In [ ]:
# pip install flask

In [1]:
from datetime import datetime

In [2]:
import base64
import io
import cv2
from imageio import imread
def process_img(shape):
    xmin=9999
    xmax=0
    ymin=9999
    ymax=0
    

    for i in range(48,59):
      
        x=int(shape.part(i).x)
        y=int(shape.part(i).y)
        if x<xmin:
            xmin=x
        if y<ymin:
            ymin=y
        if x>xmax:
            xmax=x
        if y>ymax:
            ymax=y
    return [xmin,ymin,xmax,ymax]
def draw_line(shape,b_image,i,j):
    cv2.line(b_image,(shape.part(i).x, shape.part(i).y),(shape.part(j).x, shape.part(j).y),15)
    #cv2.line(b_image,(shape.part(i).x-xmin, shape.part(i).y-ymin),(shape.part(j).x-xmin, shape.part(j).y-ymin),15)
def draw_line_loop(shape,b_image,start,end):
    for i in range(start,end):
            cv2.line(b_image,(shape.part(i).x, shape.part(i).y),(shape.part(i+1).x, shape.part(i+1).y),15)
            #cv2.line(b_image,(shape.part(i).x-xmin, shape.part(i).y-ymin),(shape.part(i+1).x-xmin, shape.part(i+1).y-ymin),15)
def get_lip(shape,origin_img,n):
    #img=origin_img.copy()
    delta=100
    lip_bounder=process_img(shape)
    img = np.zeros((lip_bounder[3]+delta,lip_bounder[2]+delta,3), np.uint8)
    img.fill(255)
    xlen=lip_bounder[2]-int(lip_bounder[0]/2)+100
    ylen=lip_bounder[3]-int(lip_bounder[1]/2)+100
    xmin=lip_bounder[0]
    ymin=lip_bounder[1]

    if n==0:
        draw_line_loop(shape,img,48,54)
        draw_line_loop(shape,img,60,64)       
        draw_line(shape,img,48,60) 
        draw_line(shape,img,64,54)
    elif n==1:
        draw_line_loop(shape,img,48,54)
        draw_line_loop(shape,img,64,67)       
        draw_line(shape,img,48,60) 
        draw_line(shape,img,64,54)
        draw_line(shape,img,67,60)
    elif n==2:
        draw_line_loop(shape,img,48,59)
        draw_line(shape,img,48,59)
    elif n==3:
        draw_line_loop(shape,img,60,67)
        draw_line(shape,img,60,67)
    elif n==4:
        draw_line_loop(shape,img,55,64)
        draw_line(shape,img,64,55)
    elif n==5:
        draw_line_loop(shape,img,55,60)
        draw_line_loop(shape,img,64,67)
        draw_line(shape,img,60,67)
        draw_line(shape,img,64,55)
    img=img[lip_bounder[1]:lip_bounder[1]+ylen+delta,lip_bounder[0]:lip_bounder[0]+ylen+delta]
    return img

class img_class:
    def __init__(self,img):
        self.img=img
    
    def get_img(self):
        return self.img
class contour_class:
    def __init__(self,con):
        self.contour=con
        
    def get_contour(self):
        return self.contour
def convert_base64(imgstr):
    img = imread(io.BytesIO(base64.b64decode(imgstr)))
    cv2_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return cv2_img

def load_template_contours():
    tnum=6
    template=[]#放template的image的array
    tcontours=[]#放template的contour的array
    for i in range(tnum):
        t=[]
        tc=[]
        for j in range(6):
            imgname='template'+str(i)+"_"+str(j)+'.jpg'
            tmp_img_class=img_class(cv2.imread(imgname))
            gray=cv2.cvtColor(tmp_img_class.get_img(), cv2.COLOR_BGR2GRAY)
            ret,tbinary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
            tmp_con, hierarchy = cv2.findContours(255-tbinary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            template_contour_class=contour_class(tmp_con)

            tc.append(template_contour_class)
            t.append(tmp_img_class)

        template.append(t)
        tcontours.append(tc)
    print("load template contours finish")
    return tcontours

In [19]:
import dlib 
import cv2
import imutils
import numpy as np
def show_img(img):
    cv2.imshow("",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def my_classifier(input_img,tcontours):
    img_num=5
    #img = cv2.imread('test'+str(img_num)+".jpg")# 縮小圖片
    # img=cv2.imread("me.jpg")
    img=input_img
    img=cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
   
    img = imutils.resize(img,width=640)
    tnum=6
    create_template_mode=0

    now = datetime.now()
    filename="uploaded/"+now.strftime("%H_%M_%S")+".jpg"
    print(filename)
    cv2.imwrite(filename,img)
    # cv2.imshow(filename,img)
    # cv2.waitKey(0)

    '''
    if create_template_mode<=0:
        for i in range(tnum):
            t=[]
            tc=[]
            for j in range(6):
                imgname='template'+str(i)+"_"+str(j)+'.jpg'
                tmp_img_class=img_class(cv2.imread(imgname))
                gray=cv2.cvtColor(tmp_img_class.get_img(), cv2.COLOR_BGR2GRAY)
                ret,tbinary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
                tmp_con, hierarchy = cv2.findContours(255-tbinary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                template_contour_class=contour_class(tmp_con)

                tc.append(template_contour_class)
                t.append(tmp_img_class)

            template.append(t)
            tcontours.append(tc)
    '''



    # Dlib 的人臉偵測器
    detector = dlib.get_frontal_face_detector()
    landmark_predictor= dlib.shape_predictor("E:\\Users\\sam\Downloads\\shape_predictor_68_face_landmarks_GTX.dat")
    # landmark_predictor= dlib.shape_predictor("C:\\Users\\sam\Downloads\\shape_predictor_68_face_landmarks_GTX.dat")
    # 偵測人臉
    faces = detector(img,1)
    print("faces count:",faces)
    if (len(faces) > 0):
        for k,d in enumerate(faces):
            cv2.rectangle(img,(d.left(),d.top()),(d.right(),d.bottom()),(255,255,255))
            shape = landmark_predictor(img,d)
            scores=[]
            lips_img=[]#裡面放的是正常img
            lips_img_with_contour=[]
            lips_contours=[]#裡面放的是contour_class物件
            for i in range(6):
                lips_img.append(get_lip(shape,img,i))
                gray = cv2.cvtColor(lips_img[i], cv2.COLOR_BGR2GRAY)
                ret,binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
                tmp_contours, hierarchy = cv2.findContours(255-binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                lips_contours.append(contour_class(tmp_contours))
                lips_img_with_contour.append(cv2.drawContours(lips_img[i].copy(),lips_contours[i].get_contour(),-1,(0,0,0),3))
                #tmp=cv2.matchShapes(tcontours[i][j].get_contour(),tmp_contours[0],1,0.0)
                # cv2.imshow("lip"+str(i),lips_img_with_contour[i])
                # cv2.waitKey(0)
                
                


                if create_template_mode>0:
                    cv2.imwrite("template"+str(img_num)+"_"+str(i)+".jpg",lips_img_with_contour[i])
                    #cv2.imshow("lip"+str(i),lips_img[i])
                    #cv2.waitKey(0)
                    #cv2.imshow("lip contour"+str(i),lips_img_with_contour[i])
                    #cv2.waitKey(0)

            cv2.destroyAllWindows()

            if create_template_mode<=0:
                for i in range(tnum):
                    score=0
                    for kk in range(6):
                        value=999
                        #test_img=template[i][k].get_img().copy()
                        #cv2.drawContours(test_img,tcontours[i][k].get_contour(),-1,(0,255,255),3)
                        #cv2.imshow("test",test_img)
                        #cv2.waitKey(0)
                        #cv2.destroyAllWindows()

                        for j in range(len(lips_contours[kk].get_contour())):
                            for l in range(len(tcontours[i][kk].get_contour())):
                                tmp=cv2.matchShapes(tcontours[i][kk].get_contour()[l],lips_contours[kk].get_contour()[j],1,0.0)
                                '''
                                bimg=np.zeros((300,300,3), np.uint8)
                                bimg.fill(255)
                                cv2.drawContours(bimg,tcontours[i][kk].get_contour(),0,(0,255,255),3)
                                cv2.imshow("con"+str(i)+"_"+str(kk)+"_"+str(l),bimg)
                                cv2.waitKey(0)
                                '''
                                if tmp<value:
                                    value=tmp
                        score+=value
                        #print(i,"-",kk,":",value)
                    scores.append(score)
                    cv2.destroyAllWindows()


            if create_template_mode<=0:
                for i in range(tnum):
                    print(i,":",scores[i])
                min_score=min(scores)
                min_idx=scores.index(min_score)
                print(min_idx,min_score)
                #cv2.imshow("image",img)
                #cv2.waitKey(0)
                #cv2.imshow("template",cv2.imread("test"+str(min_idx)+".jpg"))
                #cv2.waitKey(0)
                return min_idx
           



    
    cv2.destroyAllWindows() 
    return -1

In [ ]:
import flask
from flask import request
import cv2
import numpy as np
import time
app = flask.Flask(__name__)
template_contours=load_template_contours()


@app.route('/', methods=['GET'])
def home():
    return "<h1>Hello Flask2!</h1>"

@app.route('/uploader', methods = ['POST'])
def test():
    print("upload request")
    if request.method == 'POST':
        print("here")
        imgstr=request.get_json()['img']
        start=time.time()
        #print(imgstr)
        result=my_classifier(convert_base64(imgstr),template_contours)
        end=time.time()
        print(end-start)
        print(result)
        r={
            'result':result,
            'time':end-start
        }
    return r
          
app.run(debug=False,host="127.0.0.1")

load template contours finish
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


upload request
here
uploaded/22_38_06.jpg


C:\Users\sam\AppData\Local\Temp\ipykernel_18808\3119548310.py:84: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(io.BytesIO(base64.b64decode(imgstr)))
127.0.0.1 - - [27/Dec/2023 22:38:07] "POST /uploader HTTP/1.1" 200 -


faces count: rectangles[[(142, 194) (605, 657)]]
0 : 24.494697255271376
1 : 26.76775130705028
2 : 8.250728479387554
3 : 3.191337889710449
4 : 7.42127789370154
5 : 8.327425231374958
3 3.191337889710449
1.0015087127685547
3
upload request
here
uploaded/22_38_17.jpg


...

faces count: rectangles[[(110, 233) (664, 788)]]
0 : 21.915835960426765
1 : 24.6718691401097
2 : 8.239056060132887
3 : 4.522369068139101
4 : 0.971970544548301
5 : 1.0217033882837099
4 0.971970544548301
0.9264671802520752
4
upload request
here
uploaded/22_39_17.jpg


127.0.0.1 - - [27/Dec/2023 22:39:18] "POST /uploader HTTP/1.1" 200 -


faces count: rectangles[[(110, 49) (664, 603)]]
0 : 22.693902173013036
1 : 25.300264026933885
2 : 8.224161934690763
3 : 4.419471743463855
4 : 1.3079378217841755
5 : 1.6188027396908908
4 1.3079378217841755
1.0335659980773926
4
upload request
here
uploaded/22_39_27.jpg


127.0.0.1 - - [27/Dec/2023 22:39:28] "POST /uploader HTTP/1.1" 200 -


faces count: rectangles[[(110, 172) (664, 726)]]
0 : 24.116302066293983
1 : 25.719084931431563
2 : 9.202196999390198
3 : 4.976691132826402
4 : 1.4197492986969584
5 : 1.0234201770200801
5 1.0234201770200801
1.0415120124816895
5


In [18]:
from imageio import imread
imgstr="/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMSEhUSEhIVFRUVFxYVFRUVFxcVFRcVFxcXGBUVFRUYHSggGBolHRUVITEhJSkrLi4uFx8zODMtNygtLisBCgoKDg0OGhAQFy0fIB8rLS0tLS0rLS0tKy0rLS0rLS0tLS0tLS0tLS0tLS0tLS0tKzctLS0rNy03LS0rLS0rK//AABEIALkBEAMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAAFBgMEAQIHAAj/xAA/EAABAwIEAwUFBQcEAgMAAAABAAIDBBEFEiExBkFREyJhcYEHMpGhsVJiwdHwFCMzQnLh8TRzgsKDshUkU//EABkBAAIDAQAAAAAAAAAAAAAAAAIDAAEEBf/EACIRAAICAgIDAQEBAQAAAAAAAAABAhEDEiExBBNBUSJxI//aAAwDAQACEQMRAD8A7WvKEThbdqEFiSReUYkCyXhXZDdZUfaLYOVWQzZbNC8AtlLGRR5QzStaC5xAA3JNgF6qqWxsL3kBrRclcZ424zfUOLGHLE06DmfE+KvroLWxs4k9oMbAWU/ePN5Gg8hzXNsQx+WZxc5xd+uQ2QfMXHoN1o+bk1Dr+hppdFt85/md6DVZa4nYadf7qtFFzKsOkt5I0irZO028Vahqspvz5eCF9p4rLXoiWOPD2NmN7S42DTe5v66DdP8AQcQwSx3le2wFjdp2Px1XF4pbI1gmI5HtLvdBudr/ADCU482EuRi4owCJjWyxSAte52VoDrgWuA3y5knW/UJeoa6Vpt74G7Se96cwUzYdi8ZlDni7Qwxgu3tbugcrDXXfvFLeLBoltGLNaBbmb2F9R43S/wDRsW0HMKxUOs0Xadsp587W5ne1reqap6HPGJYyT9ob6cz5+aSMPex/cnF9rPuQQORLv+3x8Hvh5zof3cj8wce486F3UO+8PnZZppWMletgaaTLYg3adjzBG7SORULp0X4mwnKTLGLNNi5o2B6gIRFHmUUVZS5VogdUKpUuurk1IQoDF1WuGFdmeWWmCZIrla9iixhC1NOmesizA+OJWGxqy2n8FM2FT1EedAxzFsxivupx0W7KcKeoH3FDIsEIkaZQPpwq9RazDY2uUv7eoa6isCW/BCTKhoQ0HBXrY1yAtnWwmV6lUHP29Sw4hqlt8y3jnUolD9E+4BC2KA4PUPtYjRacZ41+y0rnj339xngSNT6BB9HoRvaRxWZHGCM2Yw2JH8zhuubauNzspaiTM7Xb8VC830TUWzz3k6DZSxQWWImqQv3UBo8SsE3WhctiiLRqSt2qMKaAaqEJWNVtjdFmngzOA6alEYqTPIGAbboJSobDG2Yp2G3qB8URkw/M26vmjAdG0DmSfJo/NG4KK7RpyH0WeWQ1Rx12J8UJGhvpe2+oOpHl+Kb+HZwW9jI67HWDDs5ruWo2II0Pgq9Xhwsq9HJkcQefyI5rPkdq0M0tUPdKXlropXAuA0eBYOb1t16hLFTT9k48hfUdPEfdO/heyK4XibX9w3D2+6Tuf1ayjx28kd47B7RcB2ocPsm/j9bc0O/KMyi4yplA2cFVfEq2F1Qk924I0LDuCNwD4eOvzRGQLpeO74MXlR+lAxrIiVmyzZajHZAIlkNUwC8oiWV3NWzAsvWWlRksyWqF7FYXi1VRLGGJ2caITilJY3tbqivC5Lhc8lY4jiGUO8bLFGRunGhTMaxkViwWpThZTeFew2mzOF1AWpi4dpxfMhYUVyGqOnDQFyv2vYqXTtgG0bbn+p4v9LfNdcJsF88cXVnbVc8m95HAeTTlHyCpJBtgQlYaFq4ra+iIhKXWC1JUT5Flp2VlEgWXFYC8qLPK5SDmqYV2BuyjLigvhrA0F5THgNH3TId3fRL8bcxbEPMp4oWANAGwCzZHwb8SRX3kdt3W5R5uvf6BMNNHYWQGijvlP/6SOP8Axb/YJjiCQOZFPDogGJw21HJNMjNECxKI+hVNEiwX2lwHNuC3UdUTZUteATa0g7wG3e0eD8UEY7Jdp5bdLFa0EvcfH9nvBImg6TN4Yy2Z4Oj22DvvtHuyf1dfE33JRaecaKnNJnyPtrl0dzts5p9Rf1Q+uqtV0PGnZzPKhSCZqFj9o1VGku5FcPw/M4Are5pGBYmRGoWpqkw1/DbTHdlwRz6pNqWujJDkKyp8EeKi66oXmzIYJlI2ZFsV6woyZWGv0QeKZXo5NFNivWOPDpsCFniyUNgJ+8PxQnBqzLJZa+0KvAgaOrx9CubGXw6efG9rAQq1qaxLgr1j/wCQutaMjiMrapN/DLwRdcwZWhNvDOKWFrocjpWHjjbGvijEOxge4EXyn6f4XzzUvuSfFdL44xcuZkvob6fJcwl3VQdqy5KuCMLVxWStExAs8VuwrDQtjuoUbgrAK87ZR3VFsljKIUz9QQNlFQ0t7aJqw3BAb3G9rIJySHY8cpdFPCZnBxeW3JFk0w1/dcQCLA7+SqsoAzYLcRXBHWyzzmmbYQcVyH8OjBMY+yz5mw/NGGOsEIwxpB9Air26JdhshnxBrfFCavGPuaeKuSRAL0dA12hvbpsptZeop19aCdtFWpJbyC1xdpbdOVVw5E9th3T1/NJ37EYqjIdwdPEFSaTQKfIbwiEuaLa5HOF/Ak2BB5FCuIILTCwsDy5DbbwRPhgnO/z10t8VU4onHat/W/8AhTBxIV5HXJdwml0BTLgkfeVPhyEOYEQhmEci39owjFl0Snj+FhwNgmeOraRoVBVsBBSpKuikcbq3Fjy07hatqCjvFlEA7MB4IPDSo4ybRdI2hn1RSJ+ihp8OvyRaLA5S27WkjwU3ojiG66hMcoeNuaAcbt7XI0HYX/BP+JQ5mpIr7Omt9kAfVZM69btGvBL2rVnN6uJ8ZLXD+6rCQp44tw0GMPA1b9Ekhq04MnshYjNj0lRYhkJV2DEHxahVaViuS0pLb2Rzr6BGyriNaZTcoJIEYey19OX6KDzNN9lUQp2QOXi1SZeq1AuUYo2Y1au3UnJaRt1VENpm6BaRN1Clqd7L1MzVWSrYfw1gaBflZN9FVNsND8PzSLFcmwUsRIeA4FwJsbHbXVKePY1wzKCOhGRpVWRuUqjXUXYxxyQvLibh0ZN722I6ItURHI0kEGwJHTwWaeJxdGuGRTjYQwt90Vl0CD4G3vWR+sj0QOJHLlIX6mrAuSvUmLM638iPxKFcV0z8tmA2F81t/wDG6WcCpxI8sfGOWU8ybjQ/NMxYdldgZc2rqjqNPWNdsdeh0KUeMYrTRvtobAnxCjFM6KctgJcy/u6uDTzsUV4jjzUrHndsrL+ocChdxlRdqkwPglb2b5b6g97q4XGttdfJCcZmL5bjUciNv7HwKNUuGPc45AbmPNa1yQNNBsfJDnYU4uvt+uSZ46V2J8j8HXguTMwKxjbC11+qrcIQ5DYo5jdOHtCZvRmUbdCyKws2KN0OJ9o3U6parqdzSosMqC1/yRbbIKeLXkt8SMuEDp40cxM5hdUKWNSD7AdhXC6a5HonimhDWgBK2EWuPMJtaqfLKk+CnnzRXG9lz4zWqHB2lzcE8x4J/wAMN2WS1xPhILS5ps5pzD8kefHvEX4+b1zTB2MgOiI8Ehfsad6d/aR678whTqQBxHisXjy0uJ0csN3YPwzDy5wFk6MwZvZ2I5fNUsLp7EJtjj7p8lqa2jZml/Lo5HidPlfYjY/RDax2ZOGOYeXPIHMm3mb2Sg+Le6HGw2rQKexRuCvPaFBK3ROsQ40VXbKWmHNRuUzBZqsEgOpRvBMPz6n0QiBpJCecDpw0BDJ0NxQ2Zmnwa38t1eiwhh3jJPnb8EZpI0QZAkOT+G1Rj9QPw+iItYAAfrfdXKyMAa8lfjisELxaSyi/WFJ/CLDZbPHmmrLmCR6Ka7gnjDX3aFS5YvLwkypXYeHjbVB24QAbljXeY1+O6a3hQOhVuP4DDNxTKmHUjWiwaGjoBYKvxdH/APUk7ulgeli03v8AJF4o7LXF6btKeVnVjgPOxsr145FSn/SYp4ZXlksTrA3h6jwKmiYHG+/65IBQy2e08gza19NEUpK5h1a4b7GwPwScKaTQ3OlYx0NNlIKJubfRDqCpBARGMomZwbiWHhw2SbUwGN+voumuaCEtcQYfcXRR/nkJZNlTAwbmaoI4MpVjD+h5I0KYEK2/wuNMoUZsmXDay4yk6hLssJapaGo7yHYOWNNUMNEzLceKV+KsS7KdsZ2eNPNNo0Pmk3jyjL5oCOq3nKj2UKOOznDxv8VHWAB5V2KEh59ENxMnMSuWof8ARnX9iWNF6lmATDBWgs31SCK5XMNrDfddPSonNnNuYdmpA8k22aXDza4EfNJvE1AGzOyjR4zgdCfeHxBT5g8gL232Ic0+R1/BC+N8IJiEjd47/Dw9AD6LJVSNUJHNZ49PFU5Roi5BuhtVGAdE1Emgcd1LI7urWRuq9LsEQqi1hUN3XPJOeHOSthTbJionoJmrCqGrD5Ebp9Ur0UyYKSXZJujVQQk2ShxFNqG9U25rhJvEtM4OD2i4B1tur2Arkr0vdcCnPCa8WF1zaft3kdmcrRvpqfO6YsEklLQCLOHUaX5EpE5OPI1QU+GPEtRc3CmhfdLeCwzDMJXZje4Owt0CNw3BTMc2zNkxKPCCIstK12WKQ9GOPwaStGOVPieoyUspv/I4fEJzlwZnHk5wyryxF+tyzKCMum4N7i/MIRHW2KzNMDE1hNtCQfU7+Zsh8wLXFp0I0IRYI0g/Il/Q6YLjJuBdPVNVAsDlxWlqC1wN+a6Rg1fePU8vwQZo0+AIOxxgnBC0q2hwIS3SYha4RKhrs2iS26omvIDnHZyIpTzi26E8TOt3ghlDjI5lHGLaL2pjNM64VGO4eLLWKrDhorNMAXBA4DNxgfPcMP3rKpizWmaLNbW4WJzYM/qugftAxAwPp3D7Tr/BdFHKDWI0o94ct7dEq4wywJRPhjFjU9q08hohmMvBBCzzjU7NmNtxoTZZbEqSlrbFUag6lQNk1Wr4JofMIxCxv5H804xyiVj2EA3FxfXl/j4rlOF1hGo5fROmEV92gXOZu1+nRZZx5HRF/H+G3xDtYwXRuF7DUtNtQfBKNS8A66W+K7TLOHRW2uLH4W/Ncc4opssptsdVIphufwGSEE3C0lGy0Ydd1PM3QFGAEKFuiM0xQSgfojMBS5GrF0GaR6OUc6Wqd1kUppUiRqiMJq7CyGVVQCq805tug09dqhRdBqIsBubK6yqYXAtt0O1km2kkNm+e6tUOFVBdpa4F9XEfgqcR+OMfrOk0crSNCFK+y5vJXSUsgDnAG+wde/punHDcVEzQQbq9qVGbJgp3F2GWuQP2jT5aS32nAfIo1T6lKvtQnsIGeL3n0AA+pTErMkuJo55VHW3QAKo8rMsl/wBc1GtcFSETlbN2lNmC1nct+tEohFKCaw3UmrRUXQ009Vc7q7SVmVyV6Co79kQqZrEFZ3AamEOIqnM0+SSu2I5oziVVdqXnuTsapCp9h/B8VLTYnROGHVN7FcyifYgpvwavta5VTgiJj9VD+H5pS9rfuwebvom6TXL4EJU9rf8AChP3z/6pyMZX9nIysc480bxbAw6ORwvmtcJS4YxEMYG+K6zE0Fo6EfglZUaYOmzg9VHcIWSnSvoQO0HRzvqk+obYlHGVojjTLFC/vWva6ZMHqS0g/FKsQRymm5nQ6X8+quUbLixufXDJulPFaPt8wFri7rnoNyTyUlRWkA6qiJTlPj08eqrVJFrli/LS5d+f0WRq36q1iA1VNmmvxQBm9DJY2R2nel57LHMETo6i6GSGYpfA7C9EKZ6CxSK9BKs8jZFhKR/JBqygY5xcTvyOw8ld7W69JFmCBcOw1T7K9IwRkZQOm5F1ep6ia5c1rW+JJcqIoHHkrUGHyCwubKPJH6a8UMdcouw4OybWY5na2AFgLm/qmTDqBsTbMFgqGFUxaNR6o2NkDybCM2qf8l2hOq5dx5jAmqXgatZ3B/xvmPqb/JN3FWN/slOcp/eyXazqB/M/05eJXJZXX8ytWJXyczI1tZkuvqsXWGrwWpCDYKxTvVdSQHVRkLsEtngovVPuAUvl2qLMku0JbQaIKt2iFk6olVbIYTqiiDIywotQz2QgKeF6toE7W12yV/ayf3EX+5/1KYYzoEte1d/7iIff/wCpRIy/Uc+opyCNeYXfMGfmgiPVgXz3TnULvHCk96aMdG2VZFaNCdCTjmlRKPFIta2z3DxTRxZWFtZK0dR8xdC58NL4nSDdov6c0rE6XIySb6A8aKU79ENiCLUFMX3DdT9nmfJaG19FJMrSTWLha4I19NQR4qSLUCw02/ytKmMA66fmrML2MjLnAknYIMnQ7GgPidw4jZVoDe6kr5y4knf6KGDRwBS0W+zYC2h2W8Oh0WXtWWN6IWwooIQyq5FMhkRsrTEmRqiwnHKiNI4FLzXkK7S1SGS4GxfI3UbW9EXga3olekrR1HJFIK26yydMfraGENCrYlXMp43Sye6NhzcTs0eKpPxJjBme4AePPwA5lKHGmKGX3Xd1gsG7tcxzQS6x2eHW9LeN241szJlTihf4lxIzzvkzZgdG72DeTQCNLbISCsOWzVvSpUYZOze68vLyYAeUkB1UTlvCqIbyFEKZ+iGuVulcqYSLFXshhV6qdoqKkSSMgrZhWiyERR2ildoEqe1p/wC7hH3ifkmijHdCUfayf4A/qPyURl+oQYH2IXXODMWBjay42XHWo3gGKOieNdFY5obuPcIJn7ZvMfRFOEMOD4yHbOaW7dRZG5Ym1ELXDXS6tYBR9mwWWHyJNNJDsLqLOPV9IGSujNmOa5zTf3bg2v4X/Fb0Li3YjzBTL7TcMDJmzDaQEO/qbb6i3wShTOAI1WrE9ogsJyta894m+/mguK12ptsNAiL36G29tErVT7n1VfRnUSxQAySNB63RdsLczr/zbeBGyr4NSZW9o7cgho8OZU5kvsrQJVliLTYi365KWOJFqaFkzcp94frRaOonRnXbkf1slSH4+yoINF4NIRWOFRTUqRsbPWVY1fpqcHcKGCHVGqKnSsk66G44L6b0ODMcRqR6pgocLjDgLE+ZutMPhsi1IzvjzSYtyZMstU6ELjGLLVyXFiC0taNizI2xHTUH4oHiLC09ow54za55FugsehFwD+Sdfahh5HZzjY/uz1vq4H4XXPqKYhxYbWdfR18p8Ha6efIgLoqFdHMU7XJTqKct1Bu06B34HofBatarTndm4gA5T70b9b+BI38HCy2mpwRniN2j3mn3mefVv3vjZMi/0U0VLry8vWTLANXKSMKJStURZs5TwKBTRKENqkqqrNQq4UiWzyyVgKdjVZR16gdoEme1t/fhHg4ptoNkoe1z+LD/AEFUjN9QhNKJYXEHushMaL4L7480SHHYeD22iybgJiibYIBwj7vomJmyy51dMuPAq+0elz0oPNjwfqCFyoREFdj41/0r/Nv1C5VNuPJX477RcjWJlzshcWF5py0+7fMb9PBHYNwqdX/GZ5/gmVyNT/k3rnuOgAA2A8EOyEFE59/iq0vL0RAmad2osbJrwydko7KXQnQOOx6B35pTh94I5D+SXNBwYQq8KdEdQcvVRdgm2f8A0/8A4wlqLYeQWHIq5OjhlsiGOiBROkp7c1CxEafZIk7HouU7LIhhfekP3d/M7BUIla4W3n/3Cn4I2zJ5MqiwjjmGtqIXwu0zDQ9HDVp+K4fjWDS07ssrHN6E+6fFrhoV39KntJ/0bvMfULopHMTo4xr1K3Y4tNwbEc1q7f1H1Xio0FZtI7MSbAX5Db0HJaOCysPVlGg3UzVBGp2qIhsFJEFEpo1ZEeqFXVioVcKRLZ4K7TtuFSRClUZSP//Z"
nparr = np.frombuffer(imgstr, np.uint8)
# decode image
img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
show_img(img)

TypeError: a bytes-like object is required, not 'str'

In [ ]:
pip install imageio

In [ ]:
import base64
import io
import cv2
from imageio import imread


filename = "o.jpg"
with open(filename, "rb") as fid:
    data = fid.read()

b64_bytes = base64.b64encode(data)
b64_string = b64_bytes.decode()
imgstr="/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMSEhUSEhIVFRUVFxYVFRUVFxcVFRcVFxcXGBUVFRUYHSggGBolHRUVITEhJSkrLi4uFx8zODMtNygtLisBCgoKDg0OGhAQFy0fIB8rLS0tLS0rLS0tKy0rLS0rLS0tLS0tLS0tLS0tLS0tLS0tKzctLS0rNy03LS0rLS0rK//AABEIALkBEAMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAAFBgMEAQIHAAj/xAA/EAABAwIEAwUFBQcEAgMAAAABAAIDBBEFEiExBkFREyJhcYEHMpGhsVJiwdHwFCMzQnLh8TRzgsKDshUkU//EABkBAAIDAQAAAAAAAAAAAAAAAAIDAAEEBf/EACIRAAICAgIDAQEBAQAAAAAAAAABAhEDEiExBBNBUSJxI//aAAwDAQACEQMRAD8A7WvKEThbdqEFiSReUYkCyXhXZDdZUfaLYOVWQzZbNC8AtlLGRR5QzStaC5xAA3JNgF6qqWxsL3kBrRclcZ424zfUOLGHLE06DmfE+KvroLWxs4k9oMbAWU/ePN5Gg8hzXNsQx+WZxc5xd+uQ2QfMXHoN1o+bk1Dr+hppdFt85/md6DVZa4nYadf7qtFFzKsOkt5I0irZO028Vahqspvz5eCF9p4rLXoiWOPD2NmN7S42DTe5v66DdP8AQcQwSx3le2wFjdp2Px1XF4pbI1gmI5HtLvdBudr/ADCU482EuRi4owCJjWyxSAte52VoDrgWuA3y5knW/UJeoa6Vpt74G7Se96cwUzYdi8ZlDni7Qwxgu3tbugcrDXXfvFLeLBoltGLNaBbmb2F9R43S/wDRsW0HMKxUOs0Xadsp587W5ne1reqap6HPGJYyT9ob6cz5+aSMPex/cnF9rPuQQORLv+3x8Hvh5zof3cj8wce486F3UO+8PnZZppWMletgaaTLYg3adjzBG7SORULp0X4mwnKTLGLNNi5o2B6gIRFHmUUVZS5VogdUKpUuurk1IQoDF1WuGFdmeWWmCZIrla9iixhC1NOmesizA+OJWGxqy2n8FM2FT1EedAxzFsxivupx0W7KcKeoH3FDIsEIkaZQPpwq9RazDY2uUv7eoa6isCW/BCTKhoQ0HBXrY1yAtnWwmV6lUHP29Sw4hqlt8y3jnUolD9E+4BC2KA4PUPtYjRacZ41+y0rnj339xngSNT6BB9HoRvaRxWZHGCM2Yw2JH8zhuubauNzspaiTM7Xb8VC830TUWzz3k6DZSxQWWImqQv3UBo8SsE3WhctiiLRqSt2qMKaAaqEJWNVtjdFmngzOA6alEYqTPIGAbboJSobDG2Yp2G3qB8URkw/M26vmjAdG0DmSfJo/NG4KK7RpyH0WeWQ1Rx12J8UJGhvpe2+oOpHl+Kb+HZwW9jI67HWDDs5ruWo2II0Pgq9Xhwsq9HJkcQefyI5rPkdq0M0tUPdKXlropXAuA0eBYOb1t16hLFTT9k48hfUdPEfdO/heyK4XibX9w3D2+6Tuf1ayjx28kd47B7RcB2ocPsm/j9bc0O/KMyi4yplA2cFVfEq2F1Qk924I0LDuCNwD4eOvzRGQLpeO74MXlR+lAxrIiVmyzZajHZAIlkNUwC8oiWV3NWzAsvWWlRksyWqF7FYXi1VRLGGJ2caITilJY3tbqivC5Lhc8lY4jiGUO8bLFGRunGhTMaxkViwWpThZTeFew2mzOF1AWpi4dpxfMhYUVyGqOnDQFyv2vYqXTtgG0bbn+p4v9LfNdcJsF88cXVnbVc8m95HAeTTlHyCpJBtgQlYaFq4ra+iIhKXWC1JUT5Flp2VlEgWXFYC8qLPK5SDmqYV2BuyjLigvhrA0F5THgNH3TId3fRL8bcxbEPMp4oWANAGwCzZHwb8SRX3kdt3W5R5uvf6BMNNHYWQGijvlP/6SOP8Axb/YJjiCQOZFPDogGJw21HJNMjNECxKI+hVNEiwX2lwHNuC3UdUTZUteATa0g7wG3e0eD8UEY7Jdp5bdLFa0EvcfH9nvBImg6TN4Yy2Z4Oj22DvvtHuyf1dfE33JRaecaKnNJnyPtrl0dzts5p9Rf1Q+uqtV0PGnZzPKhSCZqFj9o1VGku5FcPw/M4Are5pGBYmRGoWpqkw1/DbTHdlwRz6pNqWujJDkKyp8EeKi66oXmzIYJlI2ZFsV6woyZWGv0QeKZXo5NFNivWOPDpsCFniyUNgJ+8PxQnBqzLJZa+0KvAgaOrx9CubGXw6efG9rAQq1qaxLgr1j/wCQutaMjiMrapN/DLwRdcwZWhNvDOKWFrocjpWHjjbGvijEOxge4EXyn6f4XzzUvuSfFdL44xcuZkvob6fJcwl3VQdqy5KuCMLVxWStExAs8VuwrDQtjuoUbgrAK87ZR3VFsljKIUz9QQNlFQ0t7aJqw3BAb3G9rIJySHY8cpdFPCZnBxeW3JFk0w1/dcQCLA7+SqsoAzYLcRXBHWyzzmmbYQcVyH8OjBMY+yz5mw/NGGOsEIwxpB9Air26JdhshnxBrfFCavGPuaeKuSRAL0dA12hvbpsptZeop19aCdtFWpJbyC1xdpbdOVVw5E9th3T1/NJ37EYqjIdwdPEFSaTQKfIbwiEuaLa5HOF/Ak2BB5FCuIILTCwsDy5DbbwRPhgnO/z10t8VU4onHat/W/8AhTBxIV5HXJdwml0BTLgkfeVPhyEOYEQhmEci39owjFl0Snj+FhwNgmeOraRoVBVsBBSpKuikcbq3Fjy07hatqCjvFlEA7MB4IPDSo4ybRdI2hn1RSJ+ihp8OvyRaLA5S27WkjwU3ojiG66hMcoeNuaAcbt7XI0HYX/BP+JQ5mpIr7Omt9kAfVZM69btGvBL2rVnN6uJ8ZLXD+6rCQp44tw0GMPA1b9Ekhq04MnshYjNj0lRYhkJV2DEHxahVaViuS0pLb2Rzr6BGyriNaZTcoJIEYey19OX6KDzNN9lUQp2QOXi1SZeq1AuUYo2Y1au3UnJaRt1VENpm6BaRN1Clqd7L1MzVWSrYfw1gaBflZN9FVNsND8PzSLFcmwUsRIeA4FwJsbHbXVKePY1wzKCOhGRpVWRuUqjXUXYxxyQvLibh0ZN722I6ItURHI0kEGwJHTwWaeJxdGuGRTjYQwt90Vl0CD4G3vWR+sj0QOJHLlIX6mrAuSvUmLM638iPxKFcV0z8tmA2F81t/wDG6WcCpxI8sfGOWU8ybjQ/NMxYdldgZc2rqjqNPWNdsdeh0KUeMYrTRvtobAnxCjFM6KctgJcy/u6uDTzsUV4jjzUrHndsrL+ocChdxlRdqkwPglb2b5b6g97q4XGttdfJCcZmL5bjUciNv7HwKNUuGPc45AbmPNa1yQNNBsfJDnYU4uvt+uSZ46V2J8j8HXguTMwKxjbC11+qrcIQ5DYo5jdOHtCZvRmUbdCyKws2KN0OJ9o3U6parqdzSosMqC1/yRbbIKeLXkt8SMuEDp40cxM5hdUKWNSD7AdhXC6a5HonimhDWgBK2EWuPMJtaqfLKk+CnnzRXG9lz4zWqHB2lzcE8x4J/wAMN2WS1xPhILS5ps5pzD8kefHvEX4+b1zTB2MgOiI8Ehfsad6d/aR678whTqQBxHisXjy0uJ0csN3YPwzDy5wFk6MwZvZ2I5fNUsLp7EJtjj7p8lqa2jZml/Lo5HidPlfYjY/RDax2ZOGOYeXPIHMm3mb2Sg+Le6HGw2rQKexRuCvPaFBK3ROsQ40VXbKWmHNRuUzBZqsEgOpRvBMPz6n0QiBpJCecDpw0BDJ0NxQ2Zmnwa38t1eiwhh3jJPnb8EZpI0QZAkOT+G1Rj9QPw+iItYAAfrfdXKyMAa8lfjisELxaSyi/WFJ/CLDZbPHmmrLmCR6Ka7gnjDX3aFS5YvLwkypXYeHjbVB24QAbljXeY1+O6a3hQOhVuP4DDNxTKmHUjWiwaGjoBYKvxdH/APUk7ulgeli03v8AJF4o7LXF6btKeVnVjgPOxsr145FSn/SYp4ZXlksTrA3h6jwKmiYHG+/65IBQy2e08gza19NEUpK5h1a4b7GwPwScKaTQ3OlYx0NNlIKJubfRDqCpBARGMomZwbiWHhw2SbUwGN+voumuaCEtcQYfcXRR/nkJZNlTAwbmaoI4MpVjD+h5I0KYEK2/wuNMoUZsmXDay4yk6hLssJapaGo7yHYOWNNUMNEzLceKV+KsS7KdsZ2eNPNNo0Pmk3jyjL5oCOq3nKj2UKOOznDxv8VHWAB5V2KEh59ENxMnMSuWof8ARnX9iWNF6lmATDBWgs31SCK5XMNrDfddPSonNnNuYdmpA8k22aXDza4EfNJvE1AGzOyjR4zgdCfeHxBT5g8gL232Ic0+R1/BC+N8IJiEjd47/Dw9AD6LJVSNUJHNZ49PFU5Roi5BuhtVGAdE1Emgcd1LI7urWRuq9LsEQqi1hUN3XPJOeHOSthTbJionoJmrCqGrD5Ebp9Ur0UyYKSXZJujVQQk2ShxFNqG9U25rhJvEtM4OD2i4B1tur2Arkr0vdcCnPCa8WF1zaft3kdmcrRvpqfO6YsEklLQCLOHUaX5EpE5OPI1QU+GPEtRc3CmhfdLeCwzDMJXZje4Owt0CNw3BTMc2zNkxKPCCIstK12WKQ9GOPwaStGOVPieoyUspv/I4fEJzlwZnHk5wyryxF+tyzKCMum4N7i/MIRHW2KzNMDE1hNtCQfU7+Zsh8wLXFp0I0IRYI0g/Il/Q6YLjJuBdPVNVAsDlxWlqC1wN+a6Rg1fePU8vwQZo0+AIOxxgnBC0q2hwIS3SYha4RKhrs2iS26omvIDnHZyIpTzi26E8TOt3ghlDjI5lHGLaL2pjNM64VGO4eLLWKrDhorNMAXBA4DNxgfPcMP3rKpizWmaLNbW4WJzYM/qugftAxAwPp3D7Tr/BdFHKDWI0o94ct7dEq4wywJRPhjFjU9q08hohmMvBBCzzjU7NmNtxoTZZbEqSlrbFUag6lQNk1Wr4JofMIxCxv5H804xyiVj2EA3FxfXl/j4rlOF1hGo5fROmEV92gXOZu1+nRZZx5HRF/H+G3xDtYwXRuF7DUtNtQfBKNS8A66W+K7TLOHRW2uLH4W/Ncc4opssptsdVIphufwGSEE3C0lGy0Ydd1PM3QFGAEKFuiM0xQSgfojMBS5GrF0GaR6OUc6Wqd1kUppUiRqiMJq7CyGVVQCq805tug09dqhRdBqIsBubK6yqYXAtt0O1km2kkNm+e6tUOFVBdpa4F9XEfgqcR+OMfrOk0crSNCFK+y5vJXSUsgDnAG+wde/punHDcVEzQQbq9qVGbJgp3F2GWuQP2jT5aS32nAfIo1T6lKvtQnsIGeL3n0AA+pTErMkuJo55VHW3QAKo8rMsl/wBc1GtcFSETlbN2lNmC1nct+tEohFKCaw3UmrRUXQ009Vc7q7SVmVyV6Co79kQqZrEFZ3AamEOIqnM0+SSu2I5oziVVdqXnuTsapCp9h/B8VLTYnROGHVN7FcyifYgpvwavta5VTgiJj9VD+H5pS9rfuwebvom6TXL4EJU9rf8AChP3z/6pyMZX9nIysc480bxbAw6ORwvmtcJS4YxEMYG+K6zE0Fo6EfglZUaYOmzg9VHcIWSnSvoQO0HRzvqk+obYlHGVojjTLFC/vWva6ZMHqS0g/FKsQRymm5nQ6X8+quUbLixufXDJulPFaPt8wFri7rnoNyTyUlRWkA6qiJTlPj08eqrVJFrli/LS5d+f0WRq36q1iA1VNmmvxQBm9DJY2R2nel57LHMETo6i6GSGYpfA7C9EKZ6CxSK9BKs8jZFhKR/JBqygY5xcTvyOw8ld7W69JFmCBcOw1T7K9IwRkZQOm5F1ep6ia5c1rW+JJcqIoHHkrUGHyCwubKPJH6a8UMdcouw4OybWY5na2AFgLm/qmTDqBsTbMFgqGFUxaNR6o2NkDybCM2qf8l2hOq5dx5jAmqXgatZ3B/xvmPqb/JN3FWN/slOcp/eyXazqB/M/05eJXJZXX8ytWJXyczI1tZkuvqsXWGrwWpCDYKxTvVdSQHVRkLsEtngovVPuAUvl2qLMku0JbQaIKt2iFk6olVbIYTqiiDIywotQz2QgKeF6toE7W12yV/ayf3EX+5/1KYYzoEte1d/7iIff/wCpRIy/Uc+opyCNeYXfMGfmgiPVgXz3TnULvHCk96aMdG2VZFaNCdCTjmlRKPFIta2z3DxTRxZWFtZK0dR8xdC58NL4nSDdov6c0rE6XIySb6A8aKU79ENiCLUFMX3DdT9nmfJaG19FJMrSTWLha4I19NQR4qSLUCw02/ytKmMA66fmrML2MjLnAknYIMnQ7GgPidw4jZVoDe6kr5y4knf6KGDRwBS0W+zYC2h2W8Oh0WXtWWN6IWwooIQyq5FMhkRsrTEmRqiwnHKiNI4FLzXkK7S1SGS4GxfI3UbW9EXga3olekrR1HJFIK26yydMfraGENCrYlXMp43Sye6NhzcTs0eKpPxJjBme4AePPwA5lKHGmKGX3Xd1gsG7tcxzQS6x2eHW9LeN241szJlTihf4lxIzzvkzZgdG72DeTQCNLbISCsOWzVvSpUYZOze68vLyYAeUkB1UTlvCqIbyFEKZ+iGuVulcqYSLFXshhV6qdoqKkSSMgrZhWiyERR2ildoEqe1p/wC7hH3ifkmijHdCUfayf4A/qPyURl+oQYH2IXXODMWBjay42XHWo3gGKOieNdFY5obuPcIJn7ZvMfRFOEMOD4yHbOaW7dRZG5Ym1ELXDXS6tYBR9mwWWHyJNNJDsLqLOPV9IGSujNmOa5zTf3bg2v4X/Fb0Li3YjzBTL7TcMDJmzDaQEO/qbb6i3wShTOAI1WrE9ogsJyta894m+/mguK12ptsNAiL36G29tErVT7n1VfRnUSxQAySNB63RdsLczr/zbeBGyr4NSZW9o7cgho8OZU5kvsrQJVliLTYi365KWOJFqaFkzcp94frRaOonRnXbkf1slSH4+yoINF4NIRWOFRTUqRsbPWVY1fpqcHcKGCHVGqKnSsk66G44L6b0ODMcRqR6pgocLjDgLE+ZutMPhsi1IzvjzSYtyZMstU6ELjGLLVyXFiC0taNizI2xHTUH4oHiLC09ow54za55FugsehFwD+Sdfahh5HZzjY/uz1vq4H4XXPqKYhxYbWdfR18p8Ha6efIgLoqFdHMU7XJTqKct1Bu06B34HofBatarTndm4gA5T70b9b+BI38HCy2mpwRniN2j3mn3mefVv3vjZMi/0U0VLry8vWTLANXKSMKJStURZs5TwKBTRKENqkqqrNQq4UiWzyyVgKdjVZR16gdoEme1t/fhHg4ptoNkoe1z+LD/AEFUjN9QhNKJYXEHushMaL4L7480SHHYeD22iybgJiibYIBwj7vomJmyy51dMuPAq+0elz0oPNjwfqCFyoREFdj41/0r/Nv1C5VNuPJX477RcjWJlzshcWF5py0+7fMb9PBHYNwqdX/GZ5/gmVyNT/k3rnuOgAA2A8EOyEFE59/iq0vL0RAmad2osbJrwydko7KXQnQOOx6B35pTh94I5D+SXNBwYQq8KdEdQcvVRdgm2f8A0/8A4wlqLYeQWHIq5OjhlsiGOiBROkp7c1CxEafZIk7HouU7LIhhfekP3d/M7BUIla4W3n/3Cn4I2zJ5MqiwjjmGtqIXwu0zDQ9HDVp+K4fjWDS07ssrHN6E+6fFrhoV39KntJ/0bvMfULopHMTo4xr1K3Y4tNwbEc1q7f1H1Xio0FZtI7MSbAX5Db0HJaOCysPVlGg3UzVBGp2qIhsFJEFEpo1ZEeqFXVioVcKRLZ4K7TtuFSRClUZSP//Z"
print(b64_string[:100])
# reconstruct image as an numpy array
img = imread(io.BytesIO(base64.b64decode(imgstr)))



# finally convert RGB image to BGR for opencv
# and save result
cv2_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
show_img(cv2_img)
